In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

In [3]:
from torch.utils.data import DataLoader
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['airplane']
path = "../data/ShapeNet"

test_dataset = ShapeNet(path, "test", 2048, categories, load_renders=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

Loading (test) renders for airplane (02691156):   0%|          | 0/808 [00:00<?, ?it/s]

In [4]:
from utils.hyperparams import load_hyperparams

hparams_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/hparams.yaml'

hparams = load_hyperparams(hparams_path)

In [5]:
model_args = {
    'voxel_size' : hparams['voxel_size'],
    'nfs' : hparams['nfs'], 
    'attn_chans' : hparams['attn_chans'], 
    'attn_start' : hparams['attn_start'], 
    'cross_attn_chans' : hparams['cross_attn_chans'], 
    'cross_attn_start' : hparams['cross_attn_start'], 
    'cross_attn_cond_dim' : hparams['cross_attn_cond_dim'],
}

model = SPVUnet(**model_args)
model = GSPVD(model=model)

In [21]:
steps = 500
scheduler = 'ddim'
ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/{steps}-steps.ckpt'
ckpt = torch.load(ckpt_path, weights_only=False)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [22]:
model = model.cuda().eval()

In [23]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.visualization import display_pointclouds_grid

if scheduler == 'ddim':
    sched = DDIMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
        step_size=4,
    )
else:
    sched = DDPMSparseScheduler(
        beta_min=hparams['beta_min'], 
        beta_max=hparams['beta_max'], 
        steps=steps, 
        init_steps=hparams['n_steps'],
        mode=hparams['mode'],
    )

In [24]:
# from schedulers import create_sparse_scheduler
# sched = create_sparse_scheduler(
#     beta_min=hparams['beta_min'], 
#     beta_max=hparams['beta_max'],
#     scheduling_method="warmup" if hparams['mode'] == "warm0.1" else "linear"
# )

# gen_pc = sched.sample(model, B, cond_emb=features, mode="conditional").cuda()

In [25]:
from tqdm.auto import tqdm

all_ref_pc = []
all_gen_pc = []

mean = torch.tensor(test_loader.dataset.mean).cuda()
std = torch.tensor(test_loader.dataset.std).cuda()

i = 0
for datapoint in tqdm(test_loader):
    i += 1
    # if i > 5:
    #     continue

    ref_pc = datapoint['pc'].cuda()
    features = datapoint['render-features'].cuda()

    B, N, C = ref_pc.shape
    # gen_pc = sched.sample(model, B, cond_emb=features, mode="conditional").cuda()
    gen_pc = sched.sample(model, B, N, reference=features)

    all_ref_pc.append(ref_pc)
    all_gen_pc.append(gen_pc)

all_ref_pc = torch.cat(all_ref_pc).cuda()
all_gen_pc = torch.cat(all_gen_pc).cuda()

  0%|          | 0/26 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

...

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

Sampling:   0%|          | 0/125 [00:00<?, ?it/s]

In [26]:
from metrics.evaluation_metrics import compute_all_metrics, jsd_between_point_cloud_sets as JSD
from pprint import pprint

In [27]:
results = compute_all_metrics(all_ref_pc, all_gen_pc, batch_size=32)
results = {k: (v.cpu().detach().item()
               if not isinstance(v, float) else v) for k, v in results.items()}

pprint(results)

  0%|          | 0/808 [00:00<?, ?it/s]

  0%|          | 0/808 [00:00<?, ?it/s]

  0%|          | 0/808 [00:00<?, ?it/s]

{'1-NN-CD-acc': 0.9808168411254883,
 '1-NN-CD-acc_f': 0.9616336822509766,
 '1-NN-CD-acc_t': 1.0,
 '1-NN-EMD-acc': 0.9931930899620056,
 '1-NN-EMD-acc_f': 0.9863861203193665,
 '1-NN-EMD-acc_t': 1.0,
 'lgan_cov-CD': 0.13118812441825867,
 'lgan_cov-EMD': 0.08292078971862793,
 'lgan_mmd-CD': 0.08306979387998581,
 'lgan_mmd-EMD': 0.7008321285247803,
 'lgan_mmd_smp-CD': 0.16575732827186584,
 'lgan_mmd_smp-EMD': 1.942694067955017}


In [28]:
jsd = JSD(all_gen_pc.cpu().numpy(), all_ref_pc.cpu().numpy())
results['JSD'] = jsd
pprint('JSD: {}'.format(jsd))

'JSD: 0.14623876634591326'


In [29]:
import json
import os

folder = f'../metrics/{"-".join(categories)}/{scheduler}/no-norm'
os.makedirs(folder, exist_ok=True)

file = os.path.join(folder, f'{steps}-steps.json')
with open(file, 'w') as f:
    json.dump(results, f, indent=4)

In [30]:
def normalize_to_unit_sphere(batched_points: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Normalize a batched tensor of 3D points to the unit sphere.
    
    Args:
        batched_points: (B, N, 3) tensor, where B = batch size, N = num points.
        eps: Small value to avoid division by zero.
    
    Returns:
        (B, N, 3) tensor, where all points lie within or on the unit sphere.
    """
    # Center points by subtracting their mean (centroid)
    centroid = torch.mean(batched_points, dim=1, keepdim=True)  # (B, 1, 3)
    centered = batched_points - centroid  # (B, N, 3)

    # Find the maximum distance from the origin for each batch
    max_dist = torch.max(
        torch.sqrt(torch.sum(centered ** 2, dim=-1, keepdim=True)),  # (B, N, 1)
        dim=1, keepdim=True
    ).values  # (B, 1, 1)

    # Normalize by dividing by the maximum distance (+ eps for stability)
    normalized = centered / (max_dist + eps)  # (B, N, 3)

    return normalized

In [31]:
all_gen_pc_norm = normalize_to_unit_sphere(all_gen_pc)
all_ref_pc_norm = normalize_to_unit_sphere(all_ref_pc)

In [32]:
results = compute_all_metrics(all_ref_pc_norm, all_gen_pc_norm, batch_size=32)
results = {k: (v.cpu().detach().item()
               if not isinstance(v, float) else v) for k, v in results.items()}

pprint(results)

  0%|          | 0/808 [00:00<?, ?it/s]

  0%|          | 0/808 [00:00<?, ?it/s]

  0%|          | 0/808 [00:00<?, ?it/s]

{'1-NN-CD-acc': 0.968440592288971,
 '1-NN-CD-acc_f': 0.9368811845779419,
 '1-NN-CD-acc_t': 1.0,
 '1-NN-EMD-acc': 0.9783415794372559,
 '1-NN-EMD-acc_f': 0.9628713130950928,
 '1-NN-EMD-acc_t': 0.9938119053840637,
 'lgan_cov-CD': 0.1967821717262268,
 'lgan_cov-EMD': 0.19925741851329803,
 'lgan_mmd-CD': 0.006014720071107149,
 'lgan_mmd-EMD': 0.029273631051182747,
 'lgan_mmd_smp-CD': 0.006420005578547716,
 'lgan_mmd_smp-EMD': 0.04080174118280411}


In [33]:
jsd = JSD(all_ref_pc_norm.cpu().numpy(), all_gen_pc_norm.cpu().numpy())
results['JSD'] = jsd
pprint('JSD: {}'.format(jsd))

'JSD: 0.14853169027833601'


In [34]:
import json
import os

folder = f'../metrics/{"-".join(categories)}/{scheduler}/norm'
os.makedirs(folder, exist_ok=True)

file = os.path.join(folder, f'{steps}-steps.json')
with open(file, 'w') as f:
    json.dump(results, f, indent=4)